In [ ]:
!pip install python-chess --upgrade

  Using cached python_chess-1.999-py3-none-any.whl (1.4 kB)
  Using cached chess-1.9.3-py3-none-any.whl (148 kB)


In [ ]:
!pip install --force-reinstall chess

  Using cached chess-1.9.3-py3-none-any.whl (148 kB)
  Attempting uninstall: chess
    Found existing installation: chess 1.9.3
    Uninstalling chess-1.9.3:
      Successfully uninstalled chess-1.9.3


In [ ]:
!pip install stockfish

  Using cached stockfish-3.28.0-py3-none-any.whl (13 kB)


In [ ]:
import random
import chess
import chess.engine
import chess.svg
from stockfish import Stockfish #https://pypi.org/project/stockfish/
import os
import datetime
import multiprocessing
import time
import psutil

In [ ]:
# Pawnsdefend returns a set of all squares that are both our side and can be attacked by pawns.
def Pawnsdefend(color, myPiecesDict):
    markedTiles = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            if color and int(pos[1]) < 4:
                letter = pos[0]               
                if letter != "a":
                    left_dest = chr(ord(letter) - 1) + str(int(pos[1])+1)
                    markedTiles.add(left_dest)             
                if letter != "h":
                    right_dest = chr(ord(letter) + 1) + str(int(pos[1])+1)
                    markedTiles.add(right_dest)
            elif not color and int(pos[1]) > 5:
                letter = pos[0]               
                if letter != "a":
                    left_dest = chr(ord(letter) - 1) + str(int(pos[1])-1)
                    markedTiles.add(left_dest)             
                if letter != "h":
                    right_dest = chr(ord(letter) + 1) + str(int(pos[1])-1)
                    markedTiles.add(right_dest)
    return markedTiles

# Weakcount is the number of squares in Player A’s area that cannot be protected by Player A’s pawns
def Weakcount(markedTiles):
    return 32 - len(markedTiles)

# Enemyknightonweak is the number of knights of Player B that are in the weak squares of Player A.
def Enemyknightonweak(color, oppPiecesDict, pawn_defense_set):
    # given opp pieces dict, for each knight we find
    # add 1 to score if the knight is on our half of the board and is not on a tile in markedTiles
    score = 0
    for key, pos in oppPiecesDict.items():
        if key[0] == "n":
            if (color and int(pos[1]) <= 4) or (not color and int(pos[1]) >= 5):
                if pos not in pawn_defense_set:
                    score += 1
    return score

# Centerpawncount is the number of pawns of Player A that are in squares e4,e5,d4 or d5.
def Centerpawncount(myPiecesDict):
    good_squares = set(("e4", "e5", "d4", "d5"))
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "p" and pos in good_squares:
            score += 1
    return score

# Kingadjacent returns a set of all squares adjacent to the king's square (diagonals are adjacent).
def Kingadjacent(myPiecesDict):
    king_pos = myPiecesDict["k"]
    letter = king_pos[0]
    row = int(king_pos[1])
    markedTiles = set() 
    markedTiles.add(chr(ord(letter) - 1) + str(row+1))
    markedTiles.add(chr(ord(letter)) + str(row+1))
    markedTiles.add(chr(ord(letter) + 1) + str(row+1))
    markedTiles.add(chr(ord(letter) - 1) + str(row))
    markedTiles.add(chr(ord(letter) + 1) + str(row))
    markedTiles.add(chr(ord(letter) - 1) + str(row-1))
    markedTiles.add(chr(ord(letter)) + str(row-1))
    markedTiles.add(chr(ord(letter) + 1) + str(row-1))
    markedTiles = [elem for elem in markedTiles if elem[0].isalpha() and int(elem[1]) > 0 and int(elem[1]) < 9]
    return markedTiles

# Kingpawnshield is the number of pawns of Player A adjacent to Player A’s king.
def Kingpawnshield(myPiecesDict, kingAdjSet):
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "p" and pos in kingAdjSet:
                score += 1
    return score

# Kingattacked is the material value of the pieces of the enemy that are acting on ones king’s adjacent squares. 
def Kingattacked(board, kingAdjSet):
    # method:
    # -examine all the opponent's moves
    # -for each move, if the dest is in kingAdjSet, then add that piece's material value to the score
    # (each piece can only be added once; keep track by adding start pos to a set)
    mat_val_dict = {"P": 1, "N": 3, "B": 3, "R": 5, "Q": 9, "K": 0}
    moves = list(board.legal_moves)
    score = 0
    already_counted = set()
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            dest = strmove[2:]
            if start not in already_counted and dest in kingAdjSet:
                piece_type = board.piece_at(chess.parse_square(start)).symbol().upper()
                score += mat_val_dict[piece_type]
                already_counted.add(start)
    return score

# Kingdefended is the material value of the pieces of the Player A that are acting on Player A’s king’s adjacent squares.
def Kingdefended(board, kingAdjSet):
    # method:
    # -examine all our moves
    # -for each move, if the dest is in kingAdjSet, then add that piece's material value to the score
    # (each piece can only be added once; keep track by adding start pos to a set)
    newboard = board.copy()
    newboard.push(chess.Move.null())
    mat_val_dict = {"P": 1, "N": 3, "B": 3, "R": 5, "Q": 9, "K": 0}
    moves = list(board.legal_moves)
    score = 0
    already_counted = set()
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            dest = strmove[2:]
            if start not in already_counted and dest in kingAdjSet:
                piece_type = board.piece_at(chess.parse_square(start)).symbol().upper()
                score += mat_val_dict[piece_type]
                already_counted.add(start)
    return score

# Bishopmob is the number of squares that a bishop can go to.
def Bishopmob(board):
    newboard = board.copy()
    newboard.push(chess.Move.null())
    moves = list(board.legal_moves)
    score = 0
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            if board.piece_at(chess.parse_square(start)).symbol().upper() == "B":
                score += 1
    return score

# Bishoponlarge parameter returns 1 if the bishop on the given square is one of the two large diagonals of the board.
def Bishoponlarge(myPiecesDict):
    markedTiles = set(("a1", "b2", "c3", "d4", "e5", "f6", "g7", "h8", "a8", "b7", "c6", "d5", "e4", "f3", "g2", "h1"))
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "b" and pos in markedTiles:
        score += 1
    return score

# Bishoppair returns 1 if Player A has two or more bishops.
def Bishoppair(myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "b":
        score += 1
    return score // 2

# Knightmob is the number of squares that a specific knight can go to.
def Knightmob(board):
    newboard = board.copy()
    newboard.push(chess.Move.null())
    moves = list(board.legal_moves)
    score = 0
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            if board.piece_at(chess.parse_square(start)).symbol().upper() == "N":
                score += 1
    return score

# Knightsupport returns 1 if a given knight on a given square is supported by ones own pawn.
def Knightsupport(myPiecesDict):
    pawn_support_tiles = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            if color:
                letter = pos[0]               
                if letter != "a":
                    left_dest = chr(ord(letter) - 1) + str(int(pos[1])+1)
                    pawn_support_tiles.add(left_dest)             
                if letter != "h":
                    right_dest = chr(ord(letter) + 1) + str(int(pos[1])+1)
                    pawn_support_tiles.add(right_dest)
            else:
                letter = pos[0]               
                if letter != "a":
                    left_dest = chr(ord(letter) - 1) + str(int(pos[1])-1)
                    pawn_support_tiles.add(left_dest)             
                if letter != "h":
                    right_dest = chr(ord(letter) + 1) + str(int(pos[1])-1)
                    pawn_support_tiles.add(right_dest)
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n" and pos in pawn_support_tiles:
        score += 1
    return score

# Knightperiphery0 returns 1 if a given knight is on the squares a1 to a8,a8 to h8,a1 to h1 or h1 to h8.
def Knightperiphery0(myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n":
            if (pos[0] == "a" or pos[0] == "h"):
                score += 1
    return score

# Knightperiphery1 returns 1 if a given knight is on the squares b2 to b7,b7 to g7,b2 to g2 or g2 to g7.
def Knightperiphery1(myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n":
            if (pos[0] == "b" or pos[0] == "g") and int(pos[1]) in range(2, 8):
                score += 1
    return score

# Knightperiphery2 returns 1 if a given knight is on the squares c3 to c6,c6 to f6,c3 to f3 or f3 to f6.
def Knightperiphery2(myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n":
            if (pos[0] == "c" or pos[0] == "f") and int(pos[1]) in range(3, 7):
                score += 1
    return score

# Knightperiphery3 returns 1 if a given knight is on the squares e4, e5,d4 or d5. 
def Knightperiphery3(myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n":
            if (pos[0] == "d" or pos[0] == "e") and int(pos[1]) in range(4, 6):
                score += 1
    return score

# Isopawn returns 1 if a given pawn has no neighboring pawns of the same color.
def Isopawn(myPiecesDict):
    pawn_adj_tiles = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            letter = pos[0]
            row = int(pos[1])
            pawn_adj_tiles.add(chr(ord(letter) - 1) + str(row+1))
            pawn_adj_tiles.add(chr(ord(letter)) + str(row+1))
            pawn_adj_tiles.add(chr(ord(letter) + 1) + str(row+1))
            pawn_adj_tiles.add(chr(ord(letter) - 1) + str(row))
            pawn_adj_tiles.add(chr(ord(letter) + 1) + str(row))
            pawn_adj_tiles.add(chr(ord(letter) - 1) + str(row-1))
            pawn_adj_tiles.add(chr(ord(letter)) + str(row-1))
            pawn_adj_tiles.add(chr(ord(letter) + 1) + str(row-1))
    pawn_adj_tiles = [elem for elem in pawn_adj_tiles if elem[0].isalpha() and int(elem[1]) > 0 and int(elem[1]) < 9]
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "p" and pos not in pawn_adj_tiles:
            score += 1
    return score

# Doublepawn returns 1 if a pawn is doubled pawn. (meaning two pawns of the same color on the line column)
def Doublepawn(myPiecesDict):
    pawn_columns = {}
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            if pos[0] not in pawn_columns:
                pawn_columns[pos[0]] = 1
            else:
                pawn_columns[pos[0]] += 1                         
    score = 0
    for column, count in pawn_columns.items():
        if count > 1:
            score += count
    return score

# Passpawn returns 1 if for a given pawn there are no opposing pawns of the enemy on the neighboring columns and on the given pawn’s column ahead of the pawn.
def Passpawnlist(color, myPiecesDict, oppPiecesDict):
    opp_pawn_columns = {}  # dict of format: column : nearest to the side belonging to player
    for key, pos in oppPiecesDict.items():
        if key[0] == "p":
            if pos[0] not in opp_pawn_columns:
                opp_pawn_columns[pos[0]] = int(pos[1])
            elif color and int(pos[1]) > opp_pawn_columns[pos[0]]:
                opp_pawn_columns[pos[0]] = int(pos[1])
            elif not color and int(pos[1]) < opp_pawn_columns[pos[0]]:
                opp_pawn_columns[pos[0]] = int(pos[1])
    pass_pawn_list = []
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            # find the three columns we need to check
            letter = pos[0]
            columns_to_check = [chr(ord(letter))]           
            if letter != "a":
                columns_to_check += [chr(ord(letter) - 1)]
            if letter != "h":
                columns_to_check += [chr(ord(letter) + 1)]           
            depth = int(pos[1])
            failed = False
            for col in columns_to_check:
                if col in opp_pawn_columns and ((color and opp_pawn_columns[col] > depth) or (not color and opp_pawn_columns[col] < depth)):
                    failed = True
                    break
            if not failed:
                pass_pawn_list += [pos]
    return pass_pawn_list

def Passpawn(passPawnList):
    return len(passPawnList)

# Rookbehindpasspawn returns 1 if a rook of the same color is behind the passed pawn.
def Rookbehindpasspawn(color, myPiecesDict, passPawnList):
    # for each piece, if rook
    # add to score if rook is behind (relative to color) a tile in passpawnlist
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "r":           
            if color and any(pos[0] == elem[0] and ((color and int(pos[1]) < int(elem[1])) or (not color and int(pos[1]) > int(elem[1]))) for elem in passPawnList):
                score += 1
    return score

# Rankpassedpawn is the rank of the passed pawn.
def Rankpassedpawn(passPawnList):
    score = 0
    for pos in passPawnList:
        score += int(pos[1])
    return score

# Blockedpawn returns 1 if a central pawn on column e or d on its initial square is blocked by its own piece.
def Blockedpawn(color, myPiecesDict): 
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            if color and pos == "d2" and "d3" in myPiecesDict.values():
                score += 1
            elif color and pos == "e2" and "e3" in myPiecesDict.values():
                score += 1
            elif not color and pos == "d7" and "d6" in myPiecesDict.values():
                score += 1
            elif not color and pos == "e7" and "e6" in myPiecesDict.values():
                score += 1
    return score

# Blockedpassedpawn returns 1 if a passed pawn of Player A is blocked by a piece of Player B which prevents it from moving closer to promotion. 
def Blockedpassedpawn(color, passPawnList, myPiecesDict):
    score = 0
    for pawn_pos in passPawnList:
        letter = pawn_pos[0]
        depth = int(pawn_pos[1])        
        if color and any(elem[0] == letter and int(elem[1]) > depth for elem in myPiecesDict.values()):
            score += 1
        elif not color and any(elem[0] == letter and int(elem[1]) < depth for elem in myPiecesDict.values()):
            score += 1
    return score

# Rookopenfile returns 1 if a given rook is on a file with no pawns from either side.
def Rookopenfile(myPiecesDict, oppPiecesDict):
    file_set = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            file_set.add(pos[0])
    for key, pos in oppPiecesDict.items():
        if key[0] == "p":
            file_set.add(pos[0])
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "r" and pos[0] not in file_set:
            score += 1
    return score

# Rooksemiopenfile returns 1 if a given rook is on a file with no pawns from its own side.
def Rooksemiopenfile(myPiecesDict):
    file_set = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            file_set.add(pos[0])
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "r" and pos[0] not in file_set:
            score += 1
    return score

# Rookclosedfile returns 1 if a given rook is on a file with pawns from both sides.
def Rookclosedfile(myPiecesDict, oppPiecesDict):
    half_file_set = set()
    full_file_set = set()
    for key, pos in myPiecesDict.items():
        if key[0] == "p":
            half_file_set.add(pos[0])
    for key, pos in oppPiecesDict.items():
        if key[0] == "p" and pos[0] in half_file_set:
            full_file_set.add(pos[0])
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "r" and pos[0] in full_file_set:
            score += 1
    return score

# Rookonseventh returns 1 if a given rook is on seventh rank from the Players perspective.
def Rookonseventh(color, myPiecesDict):
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == "r":
            if (color and pos[1] == "7") or (not color and pos[1] == "2"):
                score += 1
    return score

# Rookmob is the number of squares that a specific rook can go to.
def Rookmob(board):
    newboard = board.copy()
    newboard.push(chess.Move.null())
    moves = list(board.legal_moves)
    score = 0
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            if board.piece_at(chess.parse_square(start)).symbol().upper() == "R":
                score += 1
    return score

# Queenmob is the number of squares that a specific queen can go to.
def Queenmob(board):
    newboard = board.copy()
    newboard.push(chess.Move.null())
    moves = list(board.legal_moves)
    score = 0
    for move in moves:
        strmove = move.uci()
        if len(strmove) == 4:
            start = strmove[:2]
            if board.piece_at(chess.parse_square(start)).symbol().upper() == "Q":
                score += 1
    return score

# Piececountvalue is the number of a specific piece of the Player A (valid params are: p, n, b, r, q)
def Piececountvalue(myPiecesDict, symbol):
    score = 0
    for key, pos in myPiecesDict.items():
        if key[0] == symbol:
            score += 1
    return score

In [ ]:
def assemble_piece_dict(board_position, colour):
    """
    Takes a board position and a player colour.
    Returns the piece dict for that player's colour.
    """
    p_count = 0
    kn_count = 0
    b_count = 0
    r_count = 0
    piece_dict = {}
    for i in range(0, 64):
        piece = board_position.piece_at(i)
        if piece is not None and colour == board_position.color_at(i):
            #print(piece)
            if piece.piece_type == chess.PAWN:
                piece_dict["p" + str(p_count)] = chess.SQUARE_NAMES[i]
                p_count += 1
            elif piece.piece_type == chess.KNIGHT:
                piece_dict["n" + str(kn_count)] = chess.SQUARE_NAMES[i]
                kn_count += 1
            elif piece.piece_type == chess.BISHOP:
                piece_dict["b" + str(b_count)] = chess.SQUARE_NAMES[i]
                b_count += 1
            elif piece.piece_type == chess.ROOK:
                piece_dict["r" + str(r_count)] = chess.SQUARE_NAMES[i]
                r_count += 1
            elif piece.piece_type == chess.QUEEN:
                piece_dict["q"] = chess.SQUARE_NAMES[i]
            else:  # (king)
                piece_dict["k"] = chess.SQUARE_NAMES[i]
  
    return piece_dict

In [ ]:
def individual_select_move(board, color, ind):
    """
    Given some board state, and some individual
    return the best move in the eyes of that individual.
    """
    moves = list(board.legal_moves)

    best_move = None
    best_score = -1
    for i in range(0, len(moves)):# move in moves:
        nb = board.copy()       
        nb.push(moves[i])
        
        my_pieces = assemble_piece_dict(board, color)
        opp_pieces = assemble_piece_dict(board, not color)
        
        score = score_board_position(ind, color, board, my_pieces, opp_pieces)
        score = round(score, 2)
        
        if best_score == -1 or score > best_score:
            best_score = score
            best_move = moves[i]

    return board.san(best_move)

In [ ]:
# *note: some functions require negative weights. See the ranges on page 21 in the table as a guideline.
def score_board_position(ind, color, board, my_pieces, opp_pieces):
    """
    an individual:
    [weak_count, enemy_knights_on_weak, center_pawn_count, king_pawn_shield, kingattacked_score, kingdefended_score, bishopmob_score, bishop_on_large, bishop_pair, knightmob_score, 
     knight_support, knight_periphery_0, knight_periphery_1, knight_periphery_2, knight_periphery_3, iso_pawn, double_pawn, pass_pawn, rook_behind_pass_pawn, rank_passed_pawn,
     blocked_pawn, blocked_passed_pawn, rook_open_file, rook_semi_open_file, rook_closed_file, rook_on_seventh, rookmob_score, queenmob_score, pawn_count_value, knight_count_value,
     bishop_count_value, rook_count_value, queen_count_value]
    """
    total = 0

    # helpers
    pawns_defend = Pawnsdefend(color, my_pieces)
    king_adj_set = Kingadjacent(my_pieces)
    pass_pawn_list = Passpawnlist(color, my_pieces, opp_pieces)
    
    # scoring
    weak_count = Weakcount(my_pieces) * ind[0]
    enemy_knights_on_weak = Enemyknightonweak(color, opp_pieces, pawns_defend) * ind[1]
    center_pawn_count = Centerpawncount(my_pieces) * ind[2]
    king_pawn_shield = Kingpawnshield(my_pieces, king_adj_set) * ind[3]
    kingattacked_score = Kingattacked(board, king_adj_set) * ind[4]
    kingdefended_score = Kingdefended(board, king_adj_set) * ind[5]
    bishopmob_score = Bishopmob(board) * ind[6]
    bishop_on_large = Bishoponlarge(my_pieces) * ind[7]
    bishop_pair = Bishoppair(my_pieces) * ind[8]
    knightmob_score = Knightmob(board) * ind[9]
    knight_support = Knightsupport(my_pieces) * ind[10]
    knight_periphery_0 = Knightperiphery0(my_pieces) * ind[11]
    knight_periphery_1 = Knightperiphery1(my_pieces) * ind[12]
    knight_periphery_2 = Knightperiphery2(my_pieces) * ind[13]
    knight_periphery_3 = Knightperiphery3(my_pieces) * ind[14]
    iso_pawn = Isopawn(my_pieces) * ind[15]
    double_pawn = Doublepawn(my_pieces) * ind[16]
    pass_pawn = Passpawn(pass_pawn_list) * ind[17]
    rook_behind_pass_pawn = Rookbehindpasspawn(color, my_pieces, pass_pawn_list) * ind[18]
    rank_passed_pawn = Rankpassedpawn(pass_pawn_list) * ind[19]
    blocked_pawn = Blockedpawn(color, my_pieces) * ind[20]
    blocked_passed_pawn = Blockedpassedpawn(color, pass_pawn_list, my_pieces) * ind[21]
    rook_open_file = Rookopenfile(my_pieces, opp_pieces) * ind[22]
    rook_semi_open_file = Rooksemiopenfile(my_pieces) * ind[23]
    rook_closed_file = Rookclosedfile(my_pieces, opp_pieces) * ind[24]
    rook_on_seventh = Rookonseventh(color, my_pieces) * ind[25]
    rookmob_score = Rookmob(board) * ind[26]
    queenmob_score = Queenmob(board) * ind[27]
    pawn_count_value = Piececountvalue(my_pieces, "p") * ind[28]
    knight_count_value = Piececountvalue(my_pieces, "k") * ind[29]
    bishop_count_value = Piececountvalue(my_pieces, "b") * ind[30]
    rook_count_value = Piececountvalue(my_pieces, "r") * ind[31]
    queen_count_value = Piececountvalue(my_pieces, "q") * ind[32]
    
    total = weak_count + enemy_knights_on_weak + center_pawn_count + king_pawn_shield + kingattacked_score + kingdefended_score + bishopmob_score + bishop_on_large + bishop_pair + knightmob_score 
    + knight_support + knight_periphery_0 + knight_periphery_1 + knight_periphery_2 + knight_periphery_3 + iso_pawn + double_pawn + pass_pawn + rook_behind_pass_pawn + rank_passed_pawn + blocked_pawn
    + blocked_passed_pawn + rook_open_file + rook_semi_open_file + rook_closed_file + rook_on_seventh + rookmob_score + queenmob_score + pawn_count_value + knight_count_value + bishop_count_value
    + rook_count_value + queen_count_value   
    return total

indi = [1 for i in range(33)]
color = True
board = chess.Board()
my_pieces = assemble_piece_dict(board, color)
opp_pieces = assemble_piece_dict(board, not color)
score = score_board_position(indi, color, board, my_pieces, opp_pieces)
# print(score)

move = individual_select_move(board, color, indi)
# print(move)

In [ ]:
dataset = [
  ["d4", "Nf6", "c4", "e6", "Nf3", "d5", "Nc3", "c5", "cxd5", "cxd4", "Qxd4", "exd5", "Bg5", "Be7", "e3", "O-O", "Be2", "Nc6", "Qd3", "h6", "Bh4", "Be6", "O-O", "Qb6", "Bxf6", "Bxf6", "Nxd5", "Bxd5", "Qxd5", "Qxb2", "Bc4", "Qxa1", "Rxa1", "Bxa1", "g4", "Rae8", "h4", "Re7", "g5", "hxg5", "Nxg5", "Bf6", "Qf5", "Bxg5", "hxg5", "Re5", "Qf4", "Rc5", "g6", "Ne5", "gxf7+", "Nxf7", "Be6", "Rh5", "Qc7", "g5", "Qxb7", "Rh6", "Bb3", "g4", "Qxa7", "Kg7", "e4", "Rh5", "Qc7", "Kf6", "a4"],
  ["d4", "Nf6", "c4", "g6", "Nc3", "d5", "cxd5", "Nxd5", "e4", "Nxc3", "bxc3", "Bg7", "Bb5+", "c6", "Ba4", "b5", "Bb3", "a5", "Nf3", "O-O", "O-O", "a4", "Bc2", "c5", "Rb1", "Nc6", "e5", "Rb8", "Be4", "Bf5", "Bxf5", "gxf5", "Qd3", "Qd7", "Rd1", "Rfd8", "Bf4", "cxd4", "cxd4", "e6", "Bg5", "Rdc8", "Qe3", "Ne7", "Bf6", "h6", "Qf4", "Ng6", "Qg3", "Kh7", "d5", "exd5", "Rb4", "Rc4", "Rxc4", "bxc4", "e6", "Qxe6", "Qxb8", "Bxf6", "Re1", "Qd7", "Qb6", "Kg7", "g3", "f4", "Qc5", "fxg3", "hxg3", "Nf4", "Qe3", "Nd3", "Rb1", "Qe6", "Rb7", "Qxe3", "fxe3", "c3", "Rc7", "Nb4", "Ne1", "Be5", "Rc8", "Bxg3", "Kf1", "Bxe1", "Kxe1", "Nxa2", "Kd1", "a3", "Rb8", "h5", "Kc2", "h4", "Rb1", "Kf6", "Kb3", "Nb4", "Kxb4", "a2", "Ra1", "c2", "Kc3", "h3", "Kxc2", "Kg5", "Kd3"],
  ["Nf3", "d5", "g3", "g6", "Bg2", "Bg7", "d4", "Nf6", "O-O", "O-O", "c4", "c6", "Nc3", "dxc4", "e4", "Bg4", "h3", "Bxf3", "Bxf3", "e5", "dxe5", "Nfd7", "e6", "fxe6", "Be3", "Qe7", "Bg2", "Na6", "Qe2", "Nb4", "Qxc4", "Nc2", "Nd5", "cxd5", "Qxc2", "d4", "Bd2", "Rac8", "Qb3", "Nc5", "Qa3", "Qd7", "Rac1", "b6", "e5", "Bxe5", "Rfe1", "Bg7", "b4", "Na4", "Rxc8", "Rxc8", "Qb3", "Nc3", "Rxe6", "Kh8", "Kh2", "Re8", "Rxe8+", "Qxe8", "Qc4", "h6", "a3", "a5", "bxa5", "bxa5", "Bf1", "Qf8", "Kg2", "Ne4", "Be1", "Qxa3", "Bd3", "Nd6", "Qa6", "Nf5", "h4", "h5", "Bxa5", "Kh7", "Qb5", "Qf8", "Qd5", "Qd6", "Qf3", "Qe5", "Bd8", "Qe6", "Qb7", "Nd6", "Qc6", "Qb3", "Bxg6+", "Kxg6", "Qxd6+", "Kh7", "Qd7", "Kg6", "Qc6+", "Kf7", "Qc5", "Qb7+", "f3", "Ke8", "Bc7", "Qb2+", "Kh3", "d3", "Qxh5+", "Kd7", "Qf7+", "Kc6"],
  ["e4","e5","Nf3","Nc6","d4","exd4","Nxd4","Bb4+","c3","Be7","Bf4","Nf6","e5","Nd5","Bg3","O-O","Nf5","d6","Nxe7+","Ndxe7","exd6","Nf5","Be2","Re8","O-O","Nxg3","hxg3","Qxd6","Qxd6","cxd6","Re1","d5","Nd2","d4","Bf3","Bd7","c4","Ne5","Rad1","Bc6","Bxc6","Nxc6","Rxe8+","Rxe8","Kf1","f5","Nf3","Kf7","a3","a5","b4","axb4","axb4","Nxb4","Rb1","Nc6","Rxb7+","Re7","Rb5","Kf6","Rd5","Re4","c5","g6","Kg1","Ke7","Kh2","Kf6","Kh3","h6","Kh2","Ne5","Rd6+","Kf7","Nxe5+","Rxe5","Rxd4","Rxc5","g4","fxg4","Rxg4"],
  ["d4","Nf6","c4","g6","Nc3","Bg7","e4","d6","Nge2","O-O","Ng3","e5","d5","a5","Be2","Na6","h4","h5","Bg5","Qe8","Qd2","Nc5","O-O-O","Ng4","Bxg4","Bxg4","f3","Bd7","Be3","b6","Kb1","Kh7","Qc2","a4","Nge2","f5","exf5","gxf5","Rh3","Kh8","f4","Ne4","Nxe4","fxe4","Rg3","Bg4","Rxg4","hxg4","f5","Rxf5","Ng3","Rf8","Qxe4","Qd7","a3","b5","c5","dxc5","h5","c4","h6","Bf6","Bc5","Rf7","Rf1","Re8","Bb4","Bg5","Nf5","c6","Bd6","Bf4","Ng7","Qxd6","Nxe8","Qxd5","Qxd5","cxd5","g3","Kh7","gxf4","exf4","Nd6","Rf6","Nxb5","f3","Nd4","Kxh6","Kc2","Kg5","Kd2","f2","Ne2","Rf3","Kc2","Kh4","Rh1+","Rh3","Rf1","g3","Kd2","Kg4"],
  
  ["d4","Nf6","c4","e6","Nf3","b6","g3","Ba6","Nbd2","Bb7","Bg2","Be7","O-O","O-O","b3","c5","Bb2","cxd4","Nxd4","Bxg2","Kxg2","Nc6","Nxc6","dxc6","e4","Qd3","e5","Ne4","Nf3","Rfd8","Re1","Nc5","Nd4","Rac8","Re3","Qxd1","Rxd1","Nxb3","axb3","c5","Red3","cxd4","Rxd4","Kf8","Kf3","Ke8","Ke4","Bc5","Rxd8+","Rxd8","Rxd8+","Kxd8","f3","Kc7","Bc3","Kc6","Kd3","b5","b4","Bg1","h3","Bf2","g4","Bg3","Kd4","a6","Kd3","Bf2","Bd2","Bg3","Bc3","Bf2","Bd2","Bg3","Bc3"],
  ["d4","Nf6","Nf3","d5","c4","e6","Nc3","Bb4","cxd5","exd5","Bg5","h6","Bh4","g5","Bg3","Ne4","Nd2","Nxc3","bxc3","Bxc3","Rc1","Bb2","Rxc7","Na6","Rc2","Bxd4","e3","Bg7","h4","Nb4","Rc7","O-O","hxg5","Qxg5","Bd6","Nc6","Bxf8","Kxf8","Qf3","Be6","Rxb7","Rc8","Qf4","Qxf4","exf4","Nd4","Nb3","Nxb3","axb3","Rc2","Kd1","Rxf2","Be2","Rxf4","Rxa7","Rd4+","Kc1","Rb4","Bh5","Bd4","Ra4","Be3+","Kb2","Rb6","Be2","Bd7","Ra5","Be6","Rh5","Kg7","Raxd5","Bxd5","Rxd5","Bf4","Bf3","Bd6","Kc3","Bb4+","Kc4","Be7","Rd7","Kf6","Bd5","Rb4+","Kc3","Rb6","Rb7","Rxb7"],
  ["d4","Nf6","c4","g6","Nc3","d5","Nf3","Bg7","Bg5","Ne4","Bf4","Nxc3","bxc3","c5","cxd5","cxd4","cxd4","Qxd5","e3","O-O","Be2","Nc6","O-O","Bf5","Qa4","Qa5","Qxa5","Nxa5","Rfc1","Rfc8","Nd2","Rxc1+","Rxc1","Rc8","Rxc8+","Bxc8","Bf3","Nc6","Bc7","Kf8","Nb3","Ke8","Nc5","e6","Bxc6+","bxc6","Bb8","a6","Bd6","Bf6","Kf1","Bd8","Ke2","Bb6","Na4","Bd8","Nc5","Bb6","Ne4","Kd7","Be5","Bd8","Bf6","Bb6","Ng5","c5","Nxh7","Bb7","g4","Bd5","a3","Bc4+","Kd1","Bd5","Ng5","cxd4","Bxd4","Bxd4","exd4","f6","Nh7","Ke7","g5","fxg5","Kd2","e5","Ke3","Ke6","Nf8+","Kf5","dxe5","Kxe5","Nxg6+","Kf6","Kd4","Bg2","Ne5","Kf5","Nd3","Kg4","Ke3","Kh3","Nc5","a5","f3","g4","fxg4","Kxg4","Nb3","a4","Nc5","Bc6","Kf2","Kh3","Kg1","Kg4","Nd3","Bb5","Nb4","Kh3","Na2","Be8","Nc3","Kg4","Nd1","Bb5","Nf2+","Kf3","h4","Be8","Nd1","Kg3","Nc3","Kxh4","Kf2","Kg4","Ke3","Kf5","Kd4","Ke6","Kc5","Ke7","Nxa4","Bxa4","Kb4","Be8","a4","Bxa4"],
  ["d4","Nf6","c4","g6","f3","d5","cxd5","Nxd5","e4","Nb6","Nc3","Bg7","Be3","O-O","Qd2","Nc6","O-O-O","Qd6","Nb5","Qd7","Kb1","Rd8","d5","a6","Nc3","Qe8","Qc1","Na5","Bh6","Bxh6","Qxh6","e6","Nh3","Qe7","Bd3","e5","Nf2","Nbc4","h4","Rd6","Bxc4","Nxc4","Qc1","b5","Nd3","Bd7","b3","Nb6","h5","g5","g3","c6","f4","cxd5","Nxe5","d4","Qa3","a5","Nxb5","Bxb5","Rxd4","Re6","Qxe7","Rxe7","Rc1","Nd7","Rc7","Nxe5","Rxe7","Nc6","Rd5","Bd3+","Rxd3","Nxe7","fxg5","Rb8","Rd7","Kf8","Ra7","Rb5","Ra8+","Kg7","Re8","Re5","g4","Rxe4","Kc2","Re5","Kd3","f6","gxf6+","Kxf6","Rh8","Kg7","Re8","Kh6","a3","Kg5","Rh8","h6","Rh7","Re6","Rg7+","Kf6","Rh7","Ke5","Rg7","Kf4","b4","axb4","axb4","Nc6","b5","Ne5+","Kd4","Nxg4","Kc5","Re5+","Kc6","Rxh5","b6","Ne5+","Kc7","Nc4","b7","Rc5+","Kd8","Rb5","Kc7","Rc5+","Kd8","Rb5","Kc7","Ke5"],
  ["c4","c5","Nf3","Nf6","Nc3","d5","cxd5","Nxd5","e3","Nxc3","bxc3","Qc7","d4","g6","Bb5+","Bd7","a4","Bg7","O-O","O-O","Ba3","b6","dxc5","bxc5","Qd5","Bxb5","axb5","Nd7","Rfd1","Rfd8","Ng5","e6","Qc6","Qxc6","bxc6","Ne5","c7","Rdc8","f4","h6","Ne4","Nc4","Rd7","Nb6","Rad1","Nxd7","Rxd7","Bf8","c4","a5","Nc3","a4","Nb5","Re8","e4","Rac8","Na7","Ra8","Nb5","Rac8","Na7","Ra8","Nb5"],
  
  ["d4","d5","c4","c6","cxd5","cxd5","Nc3","Nf6","Bf4","Nc6","e3","Bf5","Rc1","Rc8","Nf3","e6","Qb3","Bb4","Bb5","Bxc3+","bxc3","O-O","Bxc6","Rxc6","Qxb7","Qc8","Qxc8","Rfxc8","Ne5","Rxc3","Rxc3","Rxc3","O-O","h6","h4","Ne4","g4","Bh7","Rb1","g5","hxg5","hxg5","Bh2","Nd2","Rb8+","Kg7","Rb7","Be4","Rxf7+","Kg8","f3","Nxf3+","Rxf3","Bxf3","Nxf3","Rc1+","Kf2","Rc2+","Ke1","Rxa2","Bd6","a5","Nxg5","a4","Kd1","Rb2","Nxe6","Rb6","Nf4","Rxd6","Kc2","Rb6","Nxd5","Rb7","Nc3","a3","e4","Kf7","e5","Ke6","Kc1","Rc7","Kd2","Ra7","Na2","Rb7","Kc3","Rb8","g5","Kf5","d5","Kxe5","g6","Kxd5","g7","Ke6","g8=Q+","Rxg8","Kb3","Rg3+","Kb4","Ke5","Nc3","Re3","Kc4","Kf5","Kb4","Kg4","Na2","Kf3","Nc3","Kg2","Nd5","Rf3","Nc3","Kf1","Kc4","Ke1","Kb4","Rh3"],
  ["d4","d5","c4","c6","Nf3","Nf6","Nbd2","Bf5","Nh4","Be4","Qb3","Qb6","e3","e6","Be2","h6","Nxe4","dxe4","g3","c5","Bd2","Nc6","Bc3","cxd4","exd4","O-O-O","O-O-O","Bb4","Ng2","Bxc3","bxc3","e5","Qxb6","axb6","d5","Nb8","Ne3","Na6","g4","Nc5","h4","Ne8","h5","Nd6","Rhg1","f6","Nf5","Nxf5","gxf5","Rd7","Rg4","Rf8","Rdg1","Rff7","Kd2","Kc7","Ke3","Na4","Bd1","Nc5","Bc2","Kd8","Bxe4","Ke8","Bc2","Kf8","Rb1","Rd6","Rb5","Rc7","Rg1","Nd7","Rgb1","Rxc4","R1b3","Rh4","Bd1","Rf4","R3b4","Rxb4","cxb4","Ke7","a4","Kd8","a5","Kc7","Ba4","Kd8","Kd3","Kc7","Bb3","Kd8","Kc3","Kc7","f3","bxa5","Rxa5","Nb6","Rc5+","Kd8","Kb2","Nc8","Ra5","Kc7","Ra1","Rd7","Rg1","Nd6","Bc2","Kb6","Bd3","Rc7","Kb3","Re7","Kc3","Rc7+","Kd2","Rd7","Ke3","Re7","Rg2","Rc7","Kd2","Re7","Ke3","Rc7","Kd2","Re7"],
  ["e4","e5","Nf3","Nc6","Bb5","Nf6","d3","Bc5","Nbd2","Nd4","Nxd4","Bxd4","c3","Bb6","O-O","c6","Ba4","O-O","Bb3","d5","h3","dxe4","dxe4","Qe7","Qf3","Be6","Nc4","Nd7","Ne3","Bxe3","Bxe3","Rfd8","Rfd1","a6","Bxe6","Qxe6","Qg4","Qxg4","hxg4","h6","a4","Kf8","a5","Ke8","f3","Nf8","Bb6","Rxd1+","Rxd1","Nd7","Be3","Rd8","Kf2","f6","Ke2","Nf8","Rh1","Ne6","g3","Nf8","f4","Ng6","g5","exf4","gxf4","fxg5","fxg5","hxg5","Bxg5","Rd7","Rg1","Kf7","Be3","Nf8","Bd4","Ne6","Ke3","Rd8","Be5","Rh8","Rg3","Rh1","Rf3+","Ke7","b4","Re1+","Kd3","Rd1+","Ke2","Rh1","Ke3","Re1+","Kd3","Rd1+","Ke2","Rh1"],
  ["e4","e5","Nf3","Nc6","d4","exd4","Nxd4","Bc5","Nb3","Bb6","Nc3","Nf6","Bg5","O-O","Qf3","Nd4","Nxd4","Bxd4","O-O-O","Bxc3","bxc3","Re8","Bc4","d6","Bxf6","Qxf6","Qxf6","gxf6","Rhe1","f5","Bd5","fxe4","Bxe4","Re5","Bd5","Rxe1","Rxe1","Kf8","Re4","c6","Bb3","Bf5","Rf4","Bg6","h4","h5","Kd2","Rd8","Bc4","d5","Bd3","Kg7","a4","b6","g3","c5","Be2","Rd6","a5","Rf6","axb6","axb6","Bf3","Rd6","Bg2","Rd8","Bf3","Rd6","Bg2","Rd8","Ra4","Rd6","Rf4","b5","Bf1","Rb6","Bg2","Ra6","Bf1","Ra2","Bd3","Bxd3","Kxd3","Kg6","g4","hxg4","Rxg4+","Kf5","Rg5+","Ke6","Rg1","Ra3","Kd2","b4","cxb4","cxb4","Rg3","Ra1","Rh3","Kf6","h5","Kg7","h6+","Kh7","Rh4","Ra6","Rxb4","Kxh6","Rb5","Rf6","Ke2","Re6+","Kd3","Rf6","Rxd5","Rxf2","c4","Rf1","c5","Rc1","Kd2","Rc4","Kd3"],
  ["d4","Nf6","c4","c5","Nf3","cxd4","Nxd4","a6","Nc3","e6","e4","Qc7","a3","b6","Be3","Bb7","f3","Nc6","Rc1","Nxd4","Bxd4","Bd6","g3","h5","Qd2","h4","g4","Bf4","Be3","Bxe3","Qxe3","Qc5","Kf2","Bc6","Be2","a5","e5","Ng8","b4","axb4","axb4","Qxb4","Ra1","Ne7","Rxa8+","Bxa8","Rb1","Qa5","Qxb6","Qxb6+","Rxb6","O-O","f4","g5","fxg5","Ng6","Rb5","Bc6","Ra5","Rb8","Nb5","Nxe5","Nd4","Ng6","Ke3","Rb2","Rc5","Ne7","Ra5","Kg7","Ra7","Kg6","Nxc6","Nxc6","Rxd7","Ne5","Rd2","Rxd2","Kxd2","Kxg5","Ke3","Nd7","Bf3","f5","gxf5","Kxf5","Bc6","Ne5","Be4+","Kf6","c5","Ng4+","Kf4","Nxh2","c6","Ke7","Kg5","Kd6","Kxh4","Nf1","Kg5","Nd2","Bg2","Ke5"],

  ["e4","e5","Nf3","Nc6","Bc4","Bc5","O-O","Nf6","d3","O-O","Re1","Ng4","Re2","Kh8","h3","f5","Bg5","Nf6","Nc3","d6","Nd5","fxe4","dxe4","Be6","Nxf6","gxf6","Bxe6","fxg5","c3","Qf6","Bg4","Ne7","Qd2","Rg8","b4","Bb6","a4","a5","Qa2","Qg6","Qe6","Qg7","bxa5","Bxa5","Rb1","Ra7","Qc4","Qg6","Rb5","c6","Nxe5","dxe5","Rxe5","Rd8","Re6","Qg7","g3","Raa8","Kg2","Ng6","e5","Bc7","Bf5","Nxe5","Qb4","Rf8","Re7","Rf7","Qxb7","Raf8","Rxf7","Qxf7","Be4","h5","Bxc6","Qc4","Bb5","Qxc3","Qe4","Qc5","Rc2","Qd6","Be2","h4","gxh4","Ng6","Kf1","Bb6","Bf3","Ne5","Rd2","Qc5","Rc2","Nxf3","Rxc5","Nd2+","Ke2","Nxe4","Rc6","Bxf2","hxg5","Nxg5","a5","Ne4","Kd3","Nc5+","Kc4","Nb7","Rh6+","Kg7","Rh5","Ra8","Rf5","Nd6+"],
  ["d4","Nf6","Nf3","d5","c4","e6","Nc3","dxc4","e3","a6","a4","b6","Bxc4","Bb7","O-O","Bb4","Qe2","O-O","Rd1","Qe7","Bd3","c5","Na2","Nc6","Nxb4","Nxb4","b3","Nxd3","Rxd3","Be4","Rd1","Rfc8","Ba3","Qb7","dxc5","bxc5","Rdc1","Qxb3","Bxc5","Nd7","Bd4","Rxc1+","Rxc1","Qxa4","Ne5","Nxe5","Bxe5","Bd5","f3","f6","Bd6","Rd8","Qb2","Bb3","Qa3","Qb5","h3","e5","Rb1","Qd3","Qxb3+","Qxb3","Rxb3","Rxd6","Rb7","h5","h4","Rd3","e4","a5","Ra7","Ra3","Kf2","a4","Ra8+","Kf7","Ra7+","Kg6","Kg3","Ra2","Kh3","Ra1","g3","Ra2","f4","a3","fxe5","fxe5","g4","Ra1","g5","a2","Kg2","Kh7","Kh2","Kg6","Kg2","Re1","Ra6+","Kf7","Rxa2","Rxe4","Kh3","Rd4","Ra7+","Kg6","Re7","Re4","Kg3","Rg4+","Kh3","e4","Ra7","Rf4","Re7","Rg4","Ra7","Kf5","Rxg7","e3","Rf7+","Ke4","Re7+","Kd3","Rd7+","Kc2","Re7","Kd2","Rd7+","Ke1","Rh7","Kf2","Rf7+","Ke1","Rh7","e2","Rxh5","Re4","g6","Kd2","Rd5+","Ke3"],
  ["e4","e5","Nf3","Nc6","Bb5","a6","Ba4","Nf6","O-O","Bc5","c3","b5","Bb3","d6","a4","Bb7","d3","h6","Nh4","Rb8","axb5","axb5","Nd2","O-O","Ndf3","Ne7","h3","Bb6","Nh2","Kh8","Ng4","Nfg8","Be3","c5","Bc2","Nd5","Nf5","Nxe3","Nfxe3","g6","c4","h5","Nh2","bxc4","Nxc4","Bc7","Nf3","Kg7","Ne3","Ne7","Bb3","Bc8","Nd2","Nc6","Bd5","Nd4","Ra2","Ne6","Bxe6","Bxe6","Ndc4","Qg5","Ra7","Rfc8","Qf3","Rb3","Nd5","Bb8","Ra3","Rb7","Rfa1","Rf8","Ra6","Rd7","Rb6","f5","Qg3","Qxg3","fxg3","fxe4","dxe4","Bxd5","exd5","Bc7","Rb7","Re7","Ra3","e4","Re3","Rf5","Nd2","Rxd5","Rxe4","Rf7","Nf3","Rd1+","Re1","Rxe1+","Nxe1","d5","Rb5","Bxg3","Nf3","Ra7","Kf1","Ra1+","Ke2","Rc1","b4","cxb4","Rxb4","Rc2+","Kd3","Rxg2","Rd4","Bb8","h4","Rf2","Ke3","Ra2","Rxd5","Ba7+","Kf4","Bf2","Rd7+","Kh6","Re7","Ra4+","Re4","Ra5","Re6","Ra4+","Re4","Bg3+","Ke3","Ra3+","Ke2","Ra2+","Ke3","Kg7","Rb4","Ra3+","Ke4","Ra7","Ke3","Ra3+","Ke4","Ra6","Ke3","Re6+","Re4","Rxe4+","Kxe4","Kf6","Ke3","Kf5","Kd2","Kg4","Ke2","Kh3","Ng5+","Kxh4","Ne6","Bd6","Kf3","Kh3","Ng5+","Kh2","Ne6","Be5","Ke4","Bb8","Kf3","Bd6","Kf2","h4","Ng5","Bc5+","Kf3","Be7","Ne4","h3","Nf2","Bh4"],
  ["b3","d5","Bb2","Bf5","Nf3","e6","g3","Nf6","Bg2","Be7","O-O","O-O","c4","c6","d3","h6","Nbd2","a5","a3","Nbd7","Ra2","Bh7","Qa1","Re8","Rc1","Bd6","Ne5","Nxe5","Bxe5","Bf8","Bc3","Nd7","b4","Nb6","bxa5","Na4","Bb4","c5","Nb3","b6","Be1","bxa5","Qe5","Qb6","Rb1","Rab8","Raa1","Bd6","Qh5","Bg6","Qf3","Be5","Nd2","Qc7","Rxb8","Rxb8","Rc1","Bb2","Rc2","d4","Nb1","Ba1","Rc1","Bb2","Rc2","Nc3","Nxc3","Bxc3","Rc1","Bb2","Rb1","Bxa3","Rxb8+","Qxb8","Bd2","Bb4","Bf4","Qa7","Qc6","a4","Qc8+","Kh7","Bb8","Qe7","Qa6","a3","Be5","f6","Bf4","e5","Bc1","e4","dxe4","Bxe4","Bf1"],
  ["d4","Nf6","c4","e6","Nf3","d5","g3","Bb4+","Bd2","Be7","Bg2","O-O","O-O","Nbd7","Qb3","Nb6","c5","Nc4","Bc3","b6","c6","a5","Nbd2","Ba6","Nxc4","Bxc4","Qc2","a4","Ne5","Bb5","Rfe1","Bd6","Bd2","a3","b3","Ne4","Bxe4","dxe4","Nd7","f5","Nxf8","Qxf8","Bf4","Rd8","Be5","Bxe5","dxe5","h6","Rad1","Rxd1","Rxd1","Qb4","Kf1","e3","f3","h5","Rd7","h4","Rxc7","h3","Rd7"],

  ["d4","Nf6","c4","e6","Bf4","d5","cxd5","Nxd5","Bg3","c5","e4","Nf6","Nd2","Nc6","e5","Nd7","Ngf3","cxd4","Bc4","Be7","O-O","O-O","Rc1","Nc5","a3","a5","Nb3","b6","Nbxd4","Nxd4","Nxd4","Bb7","Nb5","Ne4","Nd6","Qb8","Qd4","b5","Bxb5","Nxg3","hxg3","Bd5","Bc6","Bxd6","exd6","Ra6","Bxd5","Rxd6","Rc5","Rxd5","Rxd5","exd5","Qxd5","Qxb2","Qxa5","g6","Qb4","Qf6","a4","Rc8","a5","h5","Qa4","Rc6","Qa1","Kg7","Rb1","Qxa1","Rxa1","Ra6","f3","Kf6","Kf2","Ke5","Ke3","Kd5","Kf4","f6","g4","hxg4","Kxg4","Ke5","f4+","Ke6","Ra2","Ke7","Kf3","Kd6","Ke4","Ke6","Ra1","Kd6","f5","gxf5+","Kxf5","Ke7","Kg6","Ke6","Re1+","Kd7","g4","Kd8","Kf7","Rd6","Re4","Rc6","Rd4+","Kc8","Rf4"],
  ["e4","e5","Nf3","Nc6","Bb5","a6","Ba4","Nf6","O-O","Be7","Re1","b5","Bb3","d6","c3","O-O","a3","h6","d4","Re8","Nbd2","Bf8","Bc2","Bg4","h3","Bh5","d5","Nb8","a4","Nbd7","Qe2","Rb8","Nb3","c6","dxc6","Nc5","axb5","axb5","Nxc5","dxc5","g4","Bg6","c4","bxc4","Bd2","Qc8","Bc3","Bd6","Nh4","Kh7","Qxc4","Re7","Red1","Qxc6","Nxg6","fxg6","Ra6","Rb6","Ba4","Rxa6","Bxc6","Rxc6","b4","Rb7","b5","Rcb6","Kg2","Bb8","Rb1","Ne8","Qxc5","Nd6","Qxe5","Nxb5","Qc5","Bd6","Qe3","g5","e5","Nxc3","Rxb6","Rxb6","Qd3+","Kh8","exd6"],
  ["e4","c6","d4","d5","e5","c5","dxc5","e6","Qg4","Nd7","Nf3","Nxc5","Be2","Ne7","O-O","Nf5","c4","dxc4","Rd1","Bd7","Bxc4","Qb6","Nc3","Be7","Nd4","h5","Qe2","Nxd4","Rxd4","Bc6","Be3","a5","Rad1","O-O","Qxh5","Qxb2","Rg4","g6","Bh6","Be8","Nd5","exd5","Bxd5","Kh7","Qh3","Rh8","Bf8+","Kg8","Rxg6+","Kxf8","Qxh8#"],
  ["d4","Nf6","c4","g6","Nc3","d5","h4","c5","cxd5","Nxd5","Na4","Nc6","e4","Nb6","d5","Ne5","h5","Nxa4","Qxa4+","Bd7","Qa3","Qb6","Qc3","Qb4","Qxb4","cxb4","f4","Ng4","hxg6","fxg6","e5","Bf5","Be2","Bg7","Bf3","O-O","Ne2","h5","Nd4","Bd7","Bd2","Rac8","Rc1","Rxc1+","Bxc1","Rc8","Ke2","Rc4","Kd3","Rc5","Be3","Bh6","g3","Nxe3","Kxe3","a5","Be4","Kg7","Nb3","Rc8","Nxa5","Ra8","Nxb7","Rxa2","Nc5","Bg4","Nd3","g5","f5","h4","gxh4","gxh4+","Kd4"],
  ["Nf3","d5","g3","g6","Bg2","Bg7","O-O","Nf6","c4","c6","b3","Ne4","d4","O-O","Bb2","Bf5","e3","a5","Nc3","Nd7","Qe2","Nxc3","Bxc3","Be4","cxd5","cxd5","Qb5","Qc7","Rfc1","Qc6","Qxc6","bxc6","Ne1","g5","Bxe4","dxe4","Rc2","Rfb8","Rac1","a4","b4","a3","Bd2","e6","Rxc6","Bf8","Nc2","Nf6","Rb1","Nd5","Rb3","g4","b5","Rb7","Bc1","Rba7","Rc5","Ra4","Bxa3","Bxc5","dxc5","e5","Nb4","Nxb4","Bxb4","Rxa2","b6","Rc2","Bc3","f6","b7"],
]
"""
totalLen = 0
for game in dataset:
    totalLen += (len(game) - 4)
print("Total number of positions in the dataset: " + str(totalLen))
"""
def assemble_fitness_set(number_of_moves):
    """
    Used to create the testing set for the agent to have its fitness scored against.
    Assembled randomly.
    Returns list of (board, fitness score after preferred move, list of position dicts (parallel to list of legal moves))
    """ 
    fitness_set = []
    engine = chess.engine.SimpleEngine.popen_uci("stockfish-5-linux/Linux/stockfish_14053109_x64")
    stop = False
    for i in range(0, len(dataset)):    
        for j in range(0, len(dataset[i]) - 4):
            if len(fitness_set) < number_of_moves:
                game = dataset[i]
                position = j
                
                board = chess.Board()
                for k in range(0, position + 1):
                    board.push_san(game[k])
                
                color = not board.turn
                # score is at this point:
                """
                print("POSITION " + str(j))
                print(board)
                """
                fitness_score = engine.analyse(board, chess.engine.Limit(time=0.1))["score"]
                
                if color:
                    fitness_val = fitness_score.white().score(mate_score=100)          
                else:
                    fitness_val = fitness_score.black().score(mate_score=100) 
                """
                print("fitness score = " + str(fitness_score))
                print("fitness val = " + str(fitness_val))
                print("\n\n")
                """
                if fitness_val == 0:
                    fitness_val = 1
                
                board.pop()
                fitness_set.append((board, fitness_val))
                
            else:
                stop = True
                break
        if stop:
            break

    engine.quit()
    del engine
    
    # print("fitness set length = " + str(len(fitness_set)))
    return fitness_set

# print(assemble_fitness_set(2))

In [ ]:
def generate_weight(index):
    """
    an individual is a list of 33 integers of the format:
    [weak_count, enemy_knights_on_weak, center_pawn_count, king_pawn_shield, kingattacked_score, kingdefended_score, bishopmob_score, bishop_on_large, bishop_pair, knightmob_score, 
     knight_support, knight_periphery_0, knight_periphery_1, knight_periphery_2, knight_periphery_3, iso_pawn, double_pawn, pass_pawn, rook_behind_pass_pawn, rank_passed_pawn,
     blocked_pawn, blocked_passed_pawn, rook_open_file, rook_semi_open_file, rook_closed_file, rook_on_seventh, rookmob_score, queenmob_score, pawn_count_value, knight_count_value,
     bishop_count_value, rook_count_value, queen_count_value]
    Each has its own weight range.
    For now, we'll just leave the range as -100 to 100
    """    
    if index == 0: # weak count
        return random.randint(-100, 0)
    elif index == 1: # enemy knights on weak
        return random.randint(-100, 0)
    elif index == 2: # center pawn count
        return random.randint(-30, 70)
    elif index == 3: # king pawn shield
        return random.randint(0, 100)
    elif index == 4: # king attacked score
        return random.randint(-100, 0)
    elif index == 5: # king defended score
        return random.randint(0, 100)
    elif index == 6: # bishop mobility score
        return random.randint(0, 15)
    elif index == 7: # bishop on large
        return random.randint(0, 100)
    elif index == 8: # bishop pair
        return random.randint(0, 100)
    elif index == 9: # knight mobility score
        return random.randint(0, 15)
    elif index == 10: # knight support
        return random.randint(0, 100)
    elif index == 11: # knight periphery 0
        return random.randint(-100, 0)
    elif index == 12: # knight periphery 1
        return random.randint(-60, 40)
    elif index == 13: # knight periphery 2
        return random.randint(-40, 60)
    elif index == 14: # knight periphery 3
        return random.randint(-30, 70)
    elif index == 15: # iso pawn
        return random.randint(-50, 50)
    elif index == 16: # double pawn
        return random.randint(-50, 0)   
    elif index == 17: # pass pawn
        return random.randint(-20, 100)
    elif index == 18: # rook behind pass pawn
        return random.randint(0, 100)
    elif index == 19: # rank passed pawn
        return random.randint(-40, 40)
    elif index == 20: # blocked pawn
        return random.randint(-50, 0)
    elif index == 21: # blocked passed pawn
        return random.randint(-50, 0)
    elif index == 22: # rook open file
        return random.randint(0, 100)
    elif index == 23: # rook semi open file
        return random.randint(0, 100)
    elif index == 24: # rook closed file
        return random.randint(-60, 30)
    elif index == 25: # rook on seventh
        return random.randint(0, 0)
    elif index == 26: # rook mobility score
        return random.randint(0, 100)
    elif index == 27: # queen mobility score
        return random.randint(0, 10)
    elif index == 28: # pawn count
        return random.randint(100, 100)
    elif index == 29: # knight count
        return random.randint(200, 400)
    elif index == 30: # bishop count
        return random.randint(220, 420)
    elif index == 31: # rook count
        return random.randint(400, 600)
    elif index == 32: # queen count
        return random.randint(800, 1000)
    else:
        return 0
def create_firstgen_agent():
    """
    Returns a weightlist of the following format:
    len = 16
    [mobility, mobility pawn weight, mobility knight weight, mobility bishop weight, mobility rook weight, mobility queen weight, mobility king weight, static, safety, territory, fortify, kings pawn shield, bishop position, knight positions, hide king, hide queen]
    """
    return [generate_weight(i) for i in range(33)]

In [ ]:
def score_fitness(individual, fitness_set, engine):
    """
    Takes an individual and the fitness set, the latter of which has the new form: list of (chess.Board: board, int: stockfish_score)
    Returns the individual's fitness (int)
    """
    # we will need a new pick move function, local to here. (the agent's one includes a whole bunch of extra stuff.)
    # takes weight list, and returns best move.
    
    #def evaluate_board(board, time_limit=0.01):
    
    def evaluate_board(board):
        # print("evaluate_board() called")
        result = engine.analyse(board, chess.engine.Limit(time=0.1))
        if "score" not in result:
            result = engine.analyse(board, chess.engine.Limit(time=0.1))
        # print("evaluate_board() returning")    
        return result["score"]
            
    score = 0.0
    for i in range(0, len(fitness_set)):
        # move-score process:
        # setup the position
        board = fitness_set[i][0]
        
        # relative scoring method
        # compare board state after player move to board state after stockfish's chosen move.
        ind_color = board.turn

        individual_board = board.copy()    
        individual_move = individual_select_move(board, ind_color, individual)
        individual_board.push_san(individual_move)
        individual_score = evaluate_board(individual_board)
        
        # print(individual_board)
        individual_val = 0
        if ind_color:
            individual_val = individual_score.white().score(mate_score=100)
        else:
            individual_val = individual_score.black().score(mate_score=100)
    
        """
        print("individual_score = " + str(individual_score))
        print("individual_val = " + str(individual_val))
        print("\n\n")
        """
            
        fitness_val = fitness_set[i][1]
        add = (individual_val / fitness_val) + 50
        score += add
        # print("score = " + str(score) + " | ind val = " + str(individual_val) + " | fitness val = " + str(fitness_val) + " | add = " + str(add))

    return score

def population_step(population, fitness_set, mp_queue=None):
    """
    Takes a single population. (list of weight lists)
    Returns the most fit. (weight list)
    """
    
    runningMax = -1
    bestIndividual = None 
    engine = chess.engine.SimpleEngine.popen_uci("stockfish-5-linux/Linux/stockfish_14053109_x64")
    
    for individual in population:
        score = score_fitness(individual, fitness_set, engine)
        if runningMax == -1 or score > runningMax:
            runningMax = score
            bestIndividual = individual
    
    # engine.close()
    engine.quit()
    del engine
    
    if mp_queue is None:
        return bestIndividual
    else:
        mp_queue.put(bestIndividual)
        

def generation_step(parent_pool, pop_cap, crossover_chance, mutation_chance, mp_queue=None):
    """
    creates a new population based on parents. (list of weight lists)
    Returns a population (list of weight lists)
    """
    new_population = []
    for i in range(0, pop_cap):
        # select 2 distinct parents
        if len(parent_pool) == 1:
            p1_index = 0
            p2_index = 0
        else:
            p1_index, p2_index = random.sample(range(0, len(parent_pool)), 2)
        # p1_index = i
        # p2_index = random.randint(0, pop_cap - 1)

        parent1 = parent_pool[p1_index]
        parent2 = parent_pool[p2_index]

        child = []

        # crossover method 1
        if random.randint(1, 100) < crossover_chance:
            crossover_point = random.randint(0, len(parent1) - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
        else:
            if random.choice([0, 1]) == 0:
                child = parent1
            else:
                child = parent2
        # one way or another, child has been assigned.
        # mutation
        for j in range(0, len(child)):
            if random.randint(1, 100) < mutation_chance:
                # perform mutation. 
                # -for now, just randomly regenerate it.
                child[j] = generate_weight(j)

        new_population.append(child)
  
    if mp_queue is None:
        return new_population
    else:
        mp_queue.put(new_population)

In [ ]:
def run(generation_number, distinct_pops, single_pop_cap, number_of_moves, crossover_chance, mutation_chance, parent_pool, starting_gen):
    """
    Performs the whole procedure.
    """
    fitness_set = assemble_fitness_set(number_of_moves)
    print("finished assembling fitness set")

    world_pop = [] # list of all populations
    
    if len(parent_pool) == 0:
        
        # clear stats records; we don't want to do this if we're resuming a job, so we put it here.
        stats_file=open("stats.txt", "w")
        stats_file.write("cleared stats file")
        stats_file.close()
        
        # we generate a random parent pool to start
        parent_pool = []
        for i in range(0, distinct_pops):
            parent_pool.append(create_firstgen_agent())
        print("creating world population from scratch")
        
    else:       
        print("creating world population from imported parent pool")

    # create world pop from parent_pool
    output = multiprocessing.Queue()
    processes = []
    for j in range(0, distinct_pops):
        p = multiprocessing.Process(target=generation_step, args=(parent_pool, single_pop_cap, crossover_chance, mutation_chance, output))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    world_pop = [output.get() for p in processes]
    print("done creating world population")
        
    generation_file=open("last_parent_pool.txt", "w")
    generation_file.write("cleared parent pool file")
    generation_file.close()

    # At this point, we have completed setup. Now, run the specified number of generations.
    for i in range(starting_gen, generation_number):        
        parent_pool.clear()
        
        output = multiprocessing.Queue()
        processes = []
        for j in range(0, distinct_pops):
            p = multiprocessing.Process(target=population_step, args=(world_pop[j], fitness_set, output))
            processes.append(p)
            p.start()
        
        # print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
        
        for p in processes:
            p.join()
        
        parent_pool = [output.get() for p in processes]
        world_pop.clear()
        
        output = multiprocessing.Queue()
        processes = []
        for j in range(0, distinct_pops):
            p = multiprocessing.Process(target=generation_step, args=(parent_pool, single_pop_cap, crossover_chance, mutation_chance, output))
            processes.append(p)
            p.start()
        for p in processes:        
            p.join() 
        world_pop = [output.get() for p in processes]
        
        engine = chess.engine.SimpleEngine.popen_uci("stockfish-5-linux/Linux/stockfish_14053109_x64")
        parent_fitness_list = [score_fitness(ind, fitness_set, engine) for ind in parent_pool]
        best_parent_fitness = max(val for val in parent_fitness_list)
        best_parent_ind = parent_pool[parent_fitness_list.index(best_parent_fitness)]
        avg_parent_fitness = sum(parent_fitness_list) / len(parent_fitness_list)
        engine.quit()
        del engine
        
        # unhashable type: list
        unique_parents_num = 0
        dummy_p = []
        for p in parent_pool:
            if p not in dummy_p:
                dummy_p += [p]
                unique_parents_num += 1        
        
        stats_file=open("stats.txt", "a")
        stats_file.write("\nGeneration " + str(i) + " | best_parent_fitness = " + str(best_parent_fitness) + " | best_parent_ind = " + str(best_parent_ind) + " | avg_parent_fitness = " + str(avg_parent_fitness) + " | unique parents = " + str(unique_parents_num))
        stats_file.close()
        
        generation_file=open("last_parent_pool.txt", "w")
        generation_file.write(str(parent_pool))
        generation_file.close()
        print("generation " + str(i) + " completed @ " + datetime.datetime.now().strftime("%H:%M:%S") + " | best parent fitness = " + str(best_parent_fitness) + " | avg_parent_fitness = " + str(avg_parent_fitness) + " | unique parents = " + str(unique_parents_num))
    
    stats_file.close()
    # At this point, we have completed all the generations and have the best individuals all stored in the parent pool.
    # return the best individual in the parent pool to end the process.
    generation_file.close()
    best = population_step(parent_pool, fitness_set)
    #best_fitness = score_fitness(best, fitness_set)
    return best #, best_fitness

In [ ]:
"""
https://www.cmpe.boun.edu.tr/~gungort/undergraduateprojects/Tuning%20of%20Chess%20Evaluation%20Function%20by%20Using%20Genetic%20Algorithms.pdf
The parameters used in that paper:
-1000 moves for fitness
-200 generations
-25% crossover rate
-0.7% mutation rate
-10-20 population size (whatever this means)

For us:
-500 moves
-200 generations
-25% crossover
-5% mutation
-30 distinct populations
-15 individuals each

"""
gen_num = 200  # the number of generations that will occur
sep_pop_num = 30  # the number of distinct populations
ind_pop_cap = 15  # the number of individuals in each population
moves_num = 1000  # the number of moves the individual is compared to during fitness scoring

# used for resuming a job.
parents=[[0, -1, 26, 56, -70, 27, 11, 22, 2, 2, 23, -92, -35, 2, -5, 26, -9, 3, 44, 31, -21, -36, 30, 50, -31, 0, 31, 5, 100, 292, 362, 489, 957], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 23, -92, 29, 2, -5, -46, -9, -14, 44, -34, -21, -36, 76, 50, -15, 0, 93, 5, 100, 380, 363, 489, 956], [0, -1, 21, 56, -70, 27, 15, 93, 75, 2, 23, -92, -35, 2, -5, -6, -23, 25, 44, -34, -18, -1, 76, 50, -31, 0, 20, 5, 100, 282, 268, 489, 957], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 96, -92, -35, 2, -5, -29, -9, -14, 44, -34, -22, -36, 76, 8, -31, 0, 72, 4, 100, 210, 268, 489, 957], [0, -1, -2, 56, -70, 27, 14, 40, 75, 2, 23, -51, -35, 2, -16, -35, -19, -14, 44, 2, -21, -36, 3, 31, -22, 0, 20, 3, 100, 210, 362, 436, 956], [0, -1, 21, 56, -22, 27, 14, 40, 75, 2, 23, -51, -35, 34, -16, -29, 0, -14, 44, 4, -26, -36, 79, 50, -31, 0, 9, 5, 100, 380, 363, 489, 956], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 23, -92, -35, 2, -5, -6, -23, 25, 44, -34, -21, -1, 76, 50, -31, 0, 20, 5, 100, 282, 362, 489, 964], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 23, -92, -35, 2, -5, -6, -23, 10, 44, -34, -21, -1, 76, 50, -31, 0, 20, 5, 100, 282, 362, 519, 959], [0, -1, 47, 56, -70, 27, 15, 99, 2, 2, 95, -92, -8, 2, -5, -29, -26, -14, 44, 31, -21, -36, 30, 50, -31, 0, 31, 5, 100, 210, 362, 489, 957], [0, -1, 21, 56, -70, 27, 6, 99, 75, 2, 96, -92, -35, 2, -5, -29, -9, -14, 44, -34, -21, -36, 76, 76, -31, 0, 52, 4, 100, 210, 268, 489, 957], [0, -1, 21, 56, -70, 27, 14, 40, 75, 2, 23, -51, -35, 2, -16, -35, 0, -14, 44, 2, -21, -36, 3, 50, -31, 0, 20, 3, 100, 210, 362, 436, 956], [0, -1, 47, 56, -70, 27, 15, 99, 55, 2, 23, -92, -35, 2, -5, -29, -9, -14, 44, 34, -21, -36, 30, 50, -31, 0, 0, 8, 100, 257, 370, 489, 957], [0, -1, 26, 56, -70, 27, 14, 99, 75, 2, 36, -92, 21, 2, 60, -2, -42, 3, 44, 11, -21, -1, 76, 50, -36, 0, 20, 5, 100, 210, 362, 489, 911], [0, -1, 47, 56, -70, 27, 15, 99, 55, 2, 23, -92, -35, 2, -5, -29, -9, 61, 44, 34, -21, -36, 30, 50, -31, 0, 0, 5, 100, 257, 370, 489, 957], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 23, -21, -35, 2, -5, -46, -9, -14, 44, -34, -21, -1, 76, 50, -31, 0, 93, 5, 100, 380, 363, 489, 956], [0, -1, 21, 56, -70, 27, 15, 99, 2, 2, 23, -92, -35, 2, -5, -29, -9, 3, 44, -34, -33, -39, 30, 50, -49, 0, 20, 4, 100, 259, 228, 489, 956], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 96, -92, -35, 2, -5, -29, -9, -14, 44, -34, -21, -36, 76, 8, -31, 0, 52, 4, 100, 210, 268, 489, 957], [0, -1, 64, 56, -70, 27, 15, 99, 2, 2, 23, -92, -35, 2, -5, -29, -9, 3, 44, -34, -33, -39, 30, 50, -31, 0, 20, 5, 100, 259, 228, 489, 956], [0, -1, 21, 56, -70, 27, 6, 99, 76, 2, 80, -92, -35, 2, -5, -29, -9, -14, 44, -34, -20, -36, 76, 76, -31, 0, 52, 4, 100, 210, 268, 489, 896], [0, -1, 47, 56, -70, 27, 15, 99, 2, 2, 95, -92, -8, 53, -5, -29, -9, -14, 44, 31, -21, -36, 30, 50, -31, 0, 31, 5, 100, 210, 362, 489, 957], [0, -1, 21, 88, -70, 27, 15, 99, 75, 2, 40, -92, -35, 2, -5, -46, -1, 84, 26, -34, -14, -36, 76, 50, -31, 0, 93, 5, 100, 380, 363, 489, 956], [0, -1, 47, 56, -70, 27, 14, 99, 75, 2, 36, -92, 21, 2, -19, -2, -5, 3, 44, 11, -21, -1, 76, 50, -54, 0, 20, 5, 100, 210, 362, 489, 911], [0, -1, 50, 56, -70, 27, 15, 99, 75, 2, 23, -92, -35, 2, -5, -46, -9, -14, 44, -34, -21, -1, 76, 50, -31, 0, 20, 5, 100, 282, 362, 489, 964], [0, -1, 21, 56, -70, 27, 14, 40, 75, 2, 23, -51, -35, 2, -16, -29, 0, -14, 44, -34, -21, -36, 79, 50, -31, 0, 9, 5, 100, 380, 363, 489, 956], [0, -1, 47, 56, -70, 27, 15, 99, 2, 2, 95, -44, -8, 2, -5, -29, -9, -14, 44, 31, -21, -36, 76, 76, -31, 0, 52, 4, 100, 210, 268, 489, 957], [0, -1, 21, 56, -70, 27, 15, 99, 75, 2, 96, -92, -35, 2, -5, -29, -9, -14, 44, -34, -21, -36, 76, 37, -31, 0, 52, 4, 100, 210, 268, 489, 957], [0, -1, 46, 56, -70, 27, 15, 99, 2, 2, 23, -92, -35, 2, -5, -29, -9, 3, 44, -34, -33, -39, 30, 50, -31, 0, 13, 5, 100, 259, 228, 489, 956], [0, -1, 26, 56, -70, 27, 14, 99, 75, 2, 36, -92, 21, 2, -25, -2, -5, 3, 44, 11, -21, -1, 64, 30, -36, 0, 20, 5, 100, 329, 362, 489, 911], [0, -1, 47, 56, -70, 27, 13, 99, 55, 2, 23, -92, -49, 2, -5, -29, -9, -14, 44, 34, -21, -48, 30, 50, -41, 0, 0, 5, 100, 257, 362, 489, 957], [0, -1, 47, 56, -70, 27, 14, 99, 2, 2, 95, -92, -8, 2, -5, -29, -9, -14, 44, 31, -21, -36, 30, 50, -31, 0, 31, 5, 100, 210, 289, 489, 957]]
starting_gen=23

# saved indi:
# best_parent_ind = [0, -1, 12, 56, -70, 27, 14, 22, 63, 2, 23, -92, -35, 2, -19, 1, -27, -14, 44, -34, -21, -36, 76, 50, -36, 0, 20, 5, 100, 210, 362, 489, 956]

# give as percentage out of 100.
# e.g. 50 would have a 1/2 chance of occuring.
crossover_chance = 20
mutation_chance = 5

print("Starting @ " + datetime.datetime.now().strftime("%H:%M:%S"))
best_individual = run(gen_num, sep_pop_num, ind_pop_cap, moves_num, crossover_chance, mutation_chance, parents, starting_gen)
print("The best individual is: " + str(best_individual))

# sanity check:
# score the fitness of the individual against a new random-generated set, to see how overfitted it is.
# (the closer the sanity fitness is to the training fitness the better)
#sanity_set = assemble_fitness_set(moves_num)
#sanity_fitness = score_fitness(best_individual, sanity_set)
#print("Sanity check fitness: " + str(sanity_fitness) + " / " + str(moves_num))

Starting @ 23:59:44
finished assembling fitness set
creating world population from imported parent pool
done creating world population
generation 23 completed @ 02:25:00 | best parent fitness = 24718.852736337893 | avg_parent_fitness = 20800.231366352877 | unique parents = 30
generation 24 completed @ 04:48:26 | best parent fitness = 27275.283130292806 | avg_parent_fitness = 19762.79392596383 | unique parents = 29
generation 25 completed @ 07:11:18 | best parent fitness = 24626.005158937085 | avg_parent_fitness = 22820.755573689068 | unique parents = 28
generation 26 completed @ 09:34:32 | best parent fitness = 24783.99582335618 | avg_parent_fitness = 21445.923145324567 | unique parents = 27
generation 27 completed @ 11:57:33 | best parent fitness = 24680.83188902024 | avg_parent_fitness = 21787.350476656004 | unique parents = 28
generation 28 completed @ 14:19:34 | best parent fitness = 27018.453046695446 | avg_parent_fitness = 21601.630700246078 | unique parents = 30
generation 29 co


KeyboardInterrupt



In [ ]:
"""
test solely fitness
"""
# individual = [68, 5, 89, 32, 57, 22, 15, 69, 29, 82, 0, 71, 7, 71, (0, 26), (18, 16)]
individual = [26, 1, 1, 1, 5, 0, 0, 9, 1, 1, 1, 22, 1, 2, (8, 4), (1, 14)]
sanity_set = assemble_fitness_set(300)
engine = chess.engine.SimpleEngine.popen_uci("stockfish-5-linux/Linux/stockfish_14053109_x64")
sanity_fitness = score_fitness(individual, sanity_set, False, engine)
print("Sanity check fitness: " + str(sanity_fitness) + " / " + str(moves_num))

In [ ]:
game_moves = []

# problem with this guy:
# just keeps swapping rook between 2 places. there must be a bug somewhere in a scoring function.
individual = [0, -1, 47, 56, -70, 27, 15, 99, 2, 2, 95, -92, -8, 2, -5, -29, -26, -14, 44, 31, -21, -36, 30, 50, -31, 0, 31, 5, 100, 210, 362, 489, 957]


game_moves = ["c4", "Nh6", "b4", "Rg8", "Nc3", "Rh8", "Bb2", "Rg8"]

# issues:

board = chess.Board()
for move in game_moves:
    board.push_san(move)

legal_moves = list(board.legal_moves)
piece_dict_list = []
for lm in legal_moves:
    dummy_board = board.copy()
    dummy_board.push(lm)
    piece_dict = assemble_piece_dict(dummy_board, not board.turn)
    piece_dict_list.append(piece_dict)

print("individual's chosen move:")
chosen_move = individual_select_move(board, board.turn, individual)
print(str(chosen_move))


# it must value mobility far too much. In fact, it hasn't been protecting its pieces at all. It let an opponent's bishop sweep right into the right flank
# maybe try setting weights myself. give it a try.

